In [1]:
! pip install datasets transformers

     |████████████████████████████████| 325 kB 8.7 MB/s 
     |████████████████████████████████| 4.0 MB 65.4 MB/s 
     |████████████████████████████████| 212 kB 53.9 MB/s 
     |████████████████████████████████| 1.1 MB 52.3 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
     |████████████████████████████████| 136 kB 69.0 MB/s 
     |████████████████████████████████| 127 kB 66.1 MB/s 
     |████████████████████████████████| 880 kB 55.5 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 78.9 MB/s 
     |████████████████████████████████| 271 kB 70.6 MB/s 
     |████████████████████████████████| 96 kB 8.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.50-py3-none-any.whl size=895166 sha256=07f98d175dc666eb30f556b7561f823f9922a2430f6fd2683a9c07881fbed8dd
  Stored in directory: /root/.cache

In [2]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:510: FutureWarning: HfApi.login: This method is deprecated in favor of `set_access_token` and will be removed in v0.7.
  FutureWarning,


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
model_checkpoint = "bert-base-multilingual-uncased" # "distilbert-base-uncased"
batch_size = 16

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from datasets import load_dataset, load_metric

In [7]:
data_files = {"train": "/content/drive/My Drive/nlp/datasets/train_pretty.json", "validation": "/content/drive/My Drive/nlp/datasets/test_pretty.json"}
#datasets = load_dataset("json", data_files="/content/drive/My Drive/nlp/datasets/dev-v2.0.json")
datasets = load_dataset("json", data_files=data_files, field="data")

Using custom data configuration default-d014794c1f8ea51a


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d014794c1f8ea51a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['context', 'id', 'answers', 'question', 'title'],
        num_rows: 86821
    })
    validation: Dataset({
        features: ['context', 'id', 'answers', 'question', 'title'],
        num_rows: 5928
    })
})

In [9]:
datasets["train"] = datasets["train"].select(list(range(0, 25000)))

In [10]:
datasets["validation"] = datasets["validation"].select(list(range(3000, 4000)))

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

In [12]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [13]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [14]:
pad_on_right = tokenizer.padding_side == "right"

In [15]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [16]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-b

In [18]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [19]:
from transformers import default_data_collator

data_collator = default_data_collator

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/Martin97Bozic/bert-base-multilingual-uncased-finetuned-squad into local empty directory.


In [21]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 26195
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4914


Epoch,Training Loss,Validation Loss
1,3.607200,3.957095
2,3.273100,4.053533
3,2.904700,4.214070


Saving model checkpoint to bert-base-multilingual-uncased-finetuned-squad/checkpoint-500
Configuration saved in bert-base-multilingual-uncased-finetuned-squad/checkpoint-500/config.json
Model weights saved in bert-base-multilingual-uncased-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-uncased-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-uncased-finetuned-squad/checkpoint-500/special_tokens_map.json
tokenizer config file saved in bert-base-multilingual-uncased-finetuned-squad/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-uncased-finetuned-squad/special_tokens_map.json
Saving model checkpoint to bert-base-multilingual-uncased-finetuned-squad/checkpoint-1000
Configuration saved in bert-base-multilingual-uncased-finetuned-squad/checkpoint-1000/config.json
Model weights saved in bert-base-multilingual-uncased-finetuned-squad/checkpoint-1000/pytorch

TrainOutput(global_step=4914, training_loss=3.3391747232079263, metrics={'train_runtime': 3192.3092, 'train_samples_per_second': 24.617, 'train_steps_per_second': 1.539, 'total_flos': 1.540050422107392e+16, 'train_loss': 3.3391747232079263, 'epoch': 3.0})

In [22]:
trainer.save_model("squad-multilingual-slo-fixed-trained")

Saving model checkpoint to squad-multilingual-slo-fixed-trained
Configuration saved in squad-multilingual-slo-fixed-trained/config.json
Model weights saved in squad-multilingual-slo-fixed-trained/pytorch_model.bin
tokenizer config file saved in squad-multilingual-slo-fixed-trained/tokenizer_config.json
Special tokens file saved in squad-multilingual-slo-fixed-trained/special_tokens_map.json
Saving model checkpoint to bert-base-multilingual-uncased-finetuned-squad
Configuration saved in bert-base-multilingual-uncased-finetuned-squad/config.json
Model weights saved in bert-base-multilingual-uncased-finetuned-squad/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-uncased-finetuned-squad/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-uncased-finetuned-squad/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/636M [00:00<?, ?B/s]

Upload file runs/May02_14-07-49_f44a7a6bedc5/events.out.tfevents.1651500487.f44a7a6bedc5.74.0:  55%|#####5    …

To https://huggingface.co/Martin97Bozic/bert-base-multilingual-uncased-finetuned-squad
   0aea9c4..c957f99  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/Martin97Bozic/bert-base-multilingual-uncased-finetuned-squad
   c957f99..2c77e9d  main -> main



## Evaluation

In [23]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [24]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [25]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 84, 199,   0, 238,  23, 154, 154,  53,   0,  77,  79, 135, 185, 205,
          27,  41], device='cuda:0'),
 tensor([ 89, 204,   0, 238,  25, 155, 155,  53, 176,  79,  66, 135, 201, 202,
          28,  44], device='cuda:0'))

In [26]:
n_best_size = 20

In [27]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [28]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [29]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1053
  Batch size = 16


In [31]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [32]:
max_answer_length = 30

In [33]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': -0.97056, 'text': 'javi s preostalim svet'},
 {'score': -1.3024888, 'text': 'javi s preostalim svetom vendar'},
 {'score': -1.4679966, 'text': 'javi'},
 {'score': -1.5801718,
  'text': 'javi s preostalim svetom vendar pa obstaja omejen element'},
 {'score': -1.604267, 'text': 'primerjavi s preostalim svet'},
 {'score': -1.6630752,
  'text': 'čajno razmeroma nizke v primerjavi s preostalim svet'},
 {'score': -1.727865, 'text': 'javi s preostalim'},
 {'score': -1.7283901, 'text': 'svet'},
 {'score': -1.7392013, 'text': 'lim svet'},
 {'score': -1.7661433,
  'text': 'javi s preostalim svetom vendar pa obstaja omejen element dr'},
 {'score': -1.8775492, 'text': 'javi s preosta'},
 {'score': -1.9021013, 'text': 'javi s preostalim svetom vendar pa ob'},
 {'score': -1.9186125, 'text': 'ostalim svet'},
 {'score': -1.9282806,
  'text': 'javi s preostalim svetom vendar pa obstaja om'},
 {'score': -1.9361957, 'text': 'primerjavi s preostalim svetom vendar'},
 {'score': -1.9403079, 'text

In [34]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [50]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    try:
                      if (
                          start_index >= len(offset_mapping)
                          or end_index >= len(offset_mapping)
                          or offset_mapping[start_index] is None
                          or offset_mapping[end_index] is None
                      ):
                          continue
                      # Don't consider answers with a length that is either < 0 or > max_answer_length.
                      if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                          continue

                      start_char = offset_mapping[start_index][0]
                      end_char = offset_mapping[end_index][1]
                      valid_answers.append(
                          {
                              "score": start_logits[start_index] + end_logits[end_index],
                              "text": context[start_char: end_char]
                          }
                      )
                    except:
                      print("error")
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)

        #if not squad_v2:
        #    predictions[example["id"]] = best_answer["text"]
        #else:
        #    answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        #    predictions[example["id"]] = answer

        #answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        #predictions[example["id"]] = answer
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [51]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 1000 example predictions split into 1053 features.


  0%|          | 0/1000 [00:00<?, ?it/s]

In [67]:
metric = load_metric("squad") #squad_v2

In [69]:
#id squad_v2 -> formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
#formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 2.7, 'f1': 24.313177531090002}

In [54]:
formatted_predictions[0:10]

[{'id': '57274d1cdd62a815002e9aaf',
  'no_answer_probability': 0.0,
  'prediction_text': 'javi s preostalim svet'},
 {'id': '57274d1cdd62a815002e9ab0',
  'no_answer_probability': 0.0,
  'prediction_text': 'približno 5 000 eur na leto'},
 {'id': '57274d1cdd62a815002e9ab1',
  'no_answer_probability': 0.0,
  'prediction_text': 'društvo'},
 {'id': '57274d1cdd62a815002e9ab2',
  'no_answer_probability': 0.0,
  'prediction_text': 'šole'},
 {'id': '57274d9bf1498d1400e8f5f6',
  'no_answer_probability': 0.0,
  'prediction_text': '1957 je vlada'},
 {'id': '57274d9bf1498d1400e8f5f7',
  'no_answer_probability': 0.0,
  'prediction_text': 'jske srednje'},
 {'id': '57274d9bf1498d1400e8f5f8',
  'no_answer_probability': 0.0,
  'prediction_text': 'jske srednje'},
 {'id': '57274d9bf1498d1400e8f5f9',
  'no_answer_probability': 0.0,
  'prediction_text': 'nacionalni'},
 {'id': '57274d9bf1498d1400e8f5fa',
  'no_answer_probability': 0.0,
  'prediction_text': 'national'},
 {'id': '57274e145951b619008f87e7',
  '

In [56]:
import json
with open("predictions.json", "w") as predictions_file:
    json.dump(formatted_predictions, predictions_file)

In [55]:
references[0:10]

[{'answers': {'answer_start': [86, 69],
   'text': ['plač učiteljev izplačuje država',
    'določeno število plač učiteljev izplačuje država']},
  'id': '57274d1cdd62a815002e9aaf'},
 {'answers': {'answer_start': [634], 'text': ['5 000 eur']},
  'id': '57274d1cdd62a815002e9ab0'},
 {'answers': {'answer_start': [333, 822],
   'text': ['mora država', 'jezusovo društvo']},
  'id': '57274d1cdd62a815002e9ab1'},
 {'answers': {'answer_start': [746, 746],
   'text': ['25 000 eur na leto', '25 000 eur']},
  'id': '57274d1cdd62a815002e9ab2'},
 {'answers': {'answer_start': [30], 'text': ['1957']},
  'id': '57274d9bf1498d1400e8f5f6'},
 {'answers': {'answer_start': [431], 'text': ['obdržale']},
  'id': '57274d9bf1498d1400e8f5f7'},
 {'answers': {'answer_start': [517], 'text': ['angleško-srednje']},
  'id': '57274d9bf1498d1400e8f5f8'},
 {'answers': {'answer_start': [282, 111],
   'text': ['vlada odgovorna', 'sistem nacionalnih']},
  'id': '57274d9bf1498d1400e8f5f9'},
 {'answers': {'answer_start': [547,

In [57]:
import json
with open("references.json", "w") as references_file:
    json.dump(references, references_file)

In [58]:
def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [65]:
compute_f1(formatted_predictions[4]['prediction_text'], references[4]['answers']['text'][0])

0.5